## Zadanie 1

In [10]:
import pandas as pd

df = pd.read_csv("spam.csv")

In [11]:
df['label'] = df['Category'].map({'ham': 0, 'spam': 1})

In [12]:
df.head()

,Category,Message,label
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [13]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['Message'])
y = df['label']

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

models = {
    "MultinomialNB": MultinomialNB(),
    "KNeighborsClassifier": KNeighborsClassifier(),
    "LogisticRegression": LogisticRegression(max_iter=1000)
}

# 6. Trenowanie i ocena
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"\n=== {name} ===")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))


=== MultinomialNB ===
Accuracy: 0.9856502242152466
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       966
           1       0.94      0.95      0.95       149

    accuracy                           0.99      1115
   macro avg       0.97      0.97      0.97      1115
weighted avg       0.99      0.99      0.99      1115


=== KNeighborsClassifier ===
Accuracy: 0.9255605381165919
              precision    recall  f1-score   support

           0       0.92      1.00      0.96       966
           1       1.00      0.44      0.61       149

    accuracy                           0.93      1115
   macro avg       0.96      0.72      0.79      1115
weighted avg       0.93      0.93      0.91      1115


=== LogisticRegression ===
Accuracy: 0.9856502242152466
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       0.99      0.90      0.94       149

    accuracy  

## Zadanie 2

In [24]:
df = pd.read_csv('IMDB Dataset.csv')

In [27]:
from sklearn.preprocessing import LabelEncoder

# 2. Konwersja etykiet tekstowych ('positive', 'negative') na numeryczne
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['sentiment'])  # 'positive' → 1, 'negative' → 0

X = df['review']
y = df['label']

In [28]:
from sklearn.pipeline import make_pipeline

models = {
    'MultinomialNB': MultinomialNB(),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'LogisticRegression': LogisticRegression(max_iter=1000)
}
scoring = ['accuracy', 'f1']

In [29]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_validate

results = {}
for name, model in models.items():
    pipeline = make_pipeline(TfidfVectorizer(), model)
    cv_result = cross_validate(
        pipeline,
        X, y,
        cv=5,
        scoring=scoring,
        return_train_score=False
    )
    results[name] = {
        'Accuracy (mean)': np.mean(cv_result['test_accuracy']),
        'F1-score (mean)': np.mean(cv_result['test_f1']),
        'Fit time (mean)': np.mean(cv_result['fit_time']),
        'Score time (mean)': np.mean(cv_result['score_time'])
    }

In [30]:
df_results = pd.DataFrame(results).T
df_results

,Accuracy (mean),F1-score (mean),Fit time (mean),Score time (mean)
MultinomialNB,0.86370,0.860267,9.814001,2.307199
KNeighborsClassifier,0.76998,0.773332,12.793124,67.689866
LogisticRegression,0.89620,0.897238,27.320832,4.379542
